# Nettoyage des Données - Filières Universitaires Tunisiennes 2025

Ce notebook nettoie et standardise les données des filières universitaires en Tunisie.


## 1. Chargement des bibliothèques

In [1]:
# Installation des packages si nécessaire (décommenter si besoin)
# install.packages(c("readxl", "dplyr", "stringr", "tidyr"))

# Chargement des bibliothèques
library(readxl)    # Pour lire les fichiers Excel
library(dplyr)     # Pour la manipulation des données
library(stringr)   # Pour la manipulation des chaînes
library(tidyr)     # Pour le nettoyage des données

cat("✅ Bibliothèques chargées avec succès\n")


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




✅ Bibliothèques chargées avec succès


## 2. Chargement des données

In [2]:
# Lecture du fichier Excel
df <- read_excel("data.xlsx", sheet = 1)

# Affichage de la structure
cat("Dimensions initiales:", nrow(df), "lignes x", ncol(df), "colonnes\n")
cat("\nNoms des colonnes originales:\n")
print(colnames(df))

Dimensions initiales: 240 lignes x 10 colonnes

Noms des colonnes originales:
 [1] "Filiere"                 "Université"             
 [3] "Etablissement"           "Gouvernorat"            
 [5] "Critères"                "Durée\r\ndes\r\nEtudes" 
 [7] "S.D.\r\nOrionté\r\n2025" "S.D.\r\nOrionté\r\n2024"
 [9] "Conditions :"            "Spécialités :"          


## 3. Nettoyage des noms de colonnes

In [3]:
# Suppression des sauts de ligne, tabulations, espaces multiples et deux-points
colnames(df) <- colnames(df) %>%
  str_replace_all("\n", " ") %>%       # Remplacer \n par espace
  str_replace_all("\t", " ") %>%       # Remplacer \t par espace
  str_replace_all(":", "") %>%         # Supprimer les :
  str_replace_all("\\s+", " ") %>%     # Remplacer espaces multiples
  str_trim()                            # Supprimer espaces début/fin

cat("Nouveaux noms de colonnes:\n")
print(colnames(df))

Nouveaux noms de colonnes:
 [1] "Filiere"           "Université"        "Etablissement"    
 [4] "Gouvernorat"       "Critères"          "Durée des Etudes" 
 [7] "S.D. Orionté 2025" "S.D. Orionté 2024" "Conditions"       
[10] "Spécialités"      


## 4. Nettoyage de la colonne 'Filiere'

In [4]:
# Suppression des lignes avec Filiere vide
df <- df %>% filter(!is.na(Filiere))

# Nettoyage de base
df <- df %>%
  mutate(Filiere = Filiere %>%
           str_replace_all("\n", " ") %>%
           str_replace_all("\\s+", " ") %>%
           str_trim() %>%
           tolower())

# Standardisation du préfixe "L EN"
df <- df %>%
  mutate(Filiere = str_replace(Filiere, "^l\\.?\\s*en\\s*", "l en "))

# Corrections d'abréviations
df <- df %>%
  mutate(Filiere = str_replace(Filiere, "l en sc\\. infirmieres", "l en sciences infirmieres"),
         Filiere = str_replace(Filiere, "l en tech\\.", "l en techniques"))

cat("Nombre de filières uniques après nettoyage:", length(unique(df$Filiere)), "\n")

Nombre de filières uniques après nettoyage: 71 


In [5]:
# Mapping des filières vers des noms cohérents
filiere_mapping <- c(
  'l en sciences de l education' = 'l en sciences de l éducation',
  'l en science et techniques des activités physiques et sportives' = 'l en sciences physiques et sportives',
  'l en physiothérapie' = 'l en santé: physiothérapie',
  'l en orthophonie' = 'l en santé: orthophonie',
  'l en nutrition humaine' = 'l en santé: nutrition',
  'l en ergothérapie' = 'l en santé: ergothérapie',
  'l en puériculture' = 'l en santé: puériculture',
  'l en sciences maïeutiques sage femme' = 'l en santé: sage-femme',
  'l en documentbibliothéconomie et archiv' = 'l en documentation et bibliothéconomie',
  'l en gestion élect de l infor et des docu' = 'l en gestion informatique documentaire',
  'l en médiation et animation' = 'l en médiation culturelle',
  'l en théa arts de la scène' = 'l en théâtre et arts de la scène',
  'prépalettre spécialité arabe' = 'prépa lettres arabe',
  'prépalettre spécialité français' = 'prépa lettres français',
  'prépalettre spécialité anglais' = 'prépa lettres anglais',
  'cycle prépasciences humaines histoire' = 'cycle prépa sciences humaines histoire',
  'cycle prépasciences humaines géographie' = 'cycle prépa sciences humaines géographie',
  'cycle prépasciences humaines philosophie' = 'cycle prépa sciences humaines philosophie',
  'l en arts du patrimoine islamique' = 'l en patrimoine islamique',
  'l en science islamique' = 'l en sciences islamiques',
  'l en géographie de lenvironnement' = 'l en géographie de l environnement',
  'l en techniques agroalimentaires et envir' = 'l en techniques agroalimentaires et environnement'
)

# Application du mapping
for (old_name in names(filiere_mapping)) {
  df <- df %>%
    mutate(Filiere = if_else(Filiere == old_name, filiere_mapping[old_name], Filiere))
}

cat("Mapping appliqué avec succès\n")

Mapping appliqué avec succès


## 5. Nettoyage de la colonne 'Université'

In [6]:
df <- df %>%
  mutate(Université = Université %>%
           str_replace_all("\n", " ") %>%
           str_trim() %>%
           tolower())

cat("Universités uniques:\n")
print(unique(df$Université))

Universités uniques:
 [1] "universite\r de tunis"        "universite de manouba"       
 [3] "universite de tunis el manar" "universite de carthage"      
 [5] "universite ez-zitouna"        "universite du sousse"        
 [7] "universite du sfax"           "universite de jendouba"      
 [9] "universite de gabes"          "universite de monastir"      
[11] "universite de kairouan"       "universite de gafsa"         


## 6. Nettoyage de la colonne 'Etablissement'

In [7]:
df <- df %>%
  mutate(Etablissement = Etablissement %>%
           str_replace_all("\n", " ") %>%
           str_replace_all("\t", " ") %>%
           str_trim() %>%
           tolower())

# Correction spécifique
df <- df %>%
  mutate(Etablissement = str_replace(Etablissement, 
                                     'faculté de médecine â€" tunis', 
                                     'faculté de médecine de tunis'))

cat("Nombre d'établissements uniques:", length(unique(df$Etablissement)), "\n")

Nombre d'établissements uniques: 83 


## 7. Nettoyage de la colonne 'Gouvernorat'

In [8]:
df <- df %>%
  mutate(Gouvernorat = Gouvernorat %>%
           str_trim() %>%
           str_to_title())  # Première lettre en majuscule

cat("Gouvernorats uniques:\n")
print(unique(df$Gouvernorat))

Gouvernorats uniques:
 [1] "Tunis"       "Zaghouane"   "Nabeul"      "Kairouan"    "Sousse"     
 [6] "Sfax"        "Jendouba"    "Le Kef"      "Beja"        "Siliana"    
[11] "Gabes"       "Mednine"     "Tataouine"   "Monastir"    "Mahdia"     
[16] "Kasserine"   "Sidi Bouzid" "Gafsa"       "Tozeur"     


## 8. Nettoyage de la colonne 'Critères'

In [9]:
df <- df %>%
  mutate(Critères = Critères %>%
           str_replace_all("\n", "") %>%
           str_trim() %>%
           toupper())

cat("Critères uniques:\n")
print(unique(df$Critères))

Critères uniques:
 [1] "FG+AR"              "FG+F"               "FG+ANG"            
 [4] "FG+\r(HG+PH)/2"     "FG+PH"              "FG+\r(F+HG)/2"     
 [7] "FG+(F+HG)/2"        "FG+(2AR+ANG+F)/4"   "FG+(AR+ANG+2F)/4"  
[10] "FG+(AR+2ANG+F)/4"   "FG+(AR+F+HG+PH)/4"  "FG+(AR+FR)/2"      
[13] "FG+ESP"             "FG+IT"              "FG+ALL"            
[16] "FG+(F+ANG+HG)/3"    "FG+(AR+ANG+F+PH)/4" "FG+(AR+ANG+F)/3"   
[19] "FG+(F+ANG+PH)/3"    "FG+INF"             "FG+SPT"            
[22] "FG+SVT"             "FG+(HG+PH)/2"       "FG+HG"             
[25] "FG+(F+ANG)/2"       "FG+(F+ANG+SPT)/3"  


## 9. Nettoyage de la colonne 'Durée des Etudes'

In [10]:
# Remplissage des valeurs manquantes
df <- df %>%
  mutate(`Durée des Etudes` = if_else(is.na(`Durée des Etudes`), "03 ans", `Durée des Etudes`))

# Uniformisation du format
df <- df %>%
  mutate(`Durée des Etudes` = `Durée des Etudes` %>%
           str_replace_all("\\s+", " ") %>%
           str_trim() %>%
           str_replace("03  ans", "03 ans"))

cat("Durées d'études uniques:\n")
print(unique(df$`Durée des Etudes`))

Durées d'études uniques:
[1] "03 ans" "02 ans" "09 ans" "06 ans"


## 10. Vérification et nettoyage des scores (2025 et 2024)

In [11]:
# Afficher les noms exacts des colonnes pour diagnostic
cat("Noms de colonnes contenant 'Orient':\n")
print(grep("Orient", colnames(df), value = TRUE, ignore.case = TRUE))

# Identifier les colonnes de scores
col_2025 <- grep("2025", colnames(df), value = TRUE)[1]
col_2024 <- grep("2024", colnames(df), value = TRUE)[1]

cat("\nColonne 2025 détectée:", col_2025, "\n")
cat("Colonne 2024 détectée:", col_2024, "\n")

Noms de colonnes contenant 'Orient':
character(0)

Colonne 2025 détectée: S.D. Orionté 2025 
Colonne 2024 détectée: S.D. Orionté 2024 


In [12]:
# Remplissage des valeurs manquantes par 0
# Utilisation de la notation avec crochets doubles pour éviter les problèmes d'accents
df[[col_2025]] <- ifelse(is.na(df[[col_2025]]), 0, df[[col_2025]])
df[[col_2024]] <- ifelse(is.na(df[[col_2024]]), 0, df[[col_2024]])

cat("\n✅ Valeurs manquantes remplacées par 0\n")

# Affichage des statistiques
cat("\nScore 2025 - min:", min(df[[col_2025]], na.rm = TRUE), 
    "max:", max(df[[col_2025]], na.rm = TRUE), "\n")
cat("Score 2024 - min:", min(df[[col_2024]], na.rm = TRUE), 
    "max:", max(df[[col_2024]], na.rm = TRUE), "\n")


✅ Valeurs manquantes remplacées par 0

Score 2025 - min: 0 max: 173.217 
Score 2024 - min: 0 max: 182.91 


## 11. Nettoyage de la colonne 'Conditions'

In [13]:
# Mapping des conditions
conditions_map <- c(
  'Rien' = 'aucune',
  '03 ans' = 'aucune',
  'Sport exigé' = 'sport obligatoire',
  'Sport exigé, sport>=16' = 'sport obligatoire >=16',
  'SVT exigée' = 'svt obligatoire',
  'Option S.V.T et Sport Obligatoires' = 'svt et sport obligatoires',
  'Filière Pour sexe Feminin et Option S.V.T. Obligatoire' = 'svt obligatoire, sexe féminin',
  'Filière Pour Sexe Feminin et Option S.V.T. Obligatoire et sport obligatoire' = 'svt et sport obligatoires, sexe féminin',
  'Filière Pour Sexe Masculin et Option S.V.T. Obligatoire et sport obligatoire' = 'svt et sport obligatoires, sexe masculin',
  'Réservée aux admis des lycées pour non-voyants, avec SVT et sport obligatoires' = 'svt et sport obligatoires, non-voyant',
  'Espagnol exigé' = 'langue obligatoire : espagnol',
  'Italien exigé' = 'langue obligatoire : italien',
  'Allemand exigé' = 'langue obligatoire : allemand',
  'Âge inférieur à 20 ans au 01 septembre' = 'âge limite',
  'age<=21 ans au premier octobre' = 'âge limite'
)

# Application du mapping
df <- df %>%
  mutate(Conditions = recode(Conditions, !!!conditions_map))

cat("Conditions uniques:\n")
print(unique(df$Conditions))

Conditions uniques:
 [1] "aucune"                                  
 [2] "sport obligatoire"                       
 [3] "âge limite"                              
 [4] "langue obligatoire : espagnol"           
 [5] "langue obligatoire : italien"            
 [6] "langue obligatoire : allemand"           
 [7] "sport obligatoire >=16"                  
 [8] "svt obligatoire"                         
 [9] "svt et sport obligatoires"               
[10] "svt obligatoire, sexe féminin"           
[11] "svt et sport obligatoires, non-voyant"   
[12] "svt et sport obligatoires, sexe féminin" 
[13] "svt et sport obligatoires, sexe masculin"


## 12. Nettoyage de la colonne 'Spécialités'

In [14]:
# Fonction pour nettoyer une spécialité
nettoyer_specialite <- function(valeur) {
  # Gérer les valeurs vides ou NA
  if (is.na(valeur) || str_trim(valeur) %in% c('', '\t/', '/')) {
    return(list())
  }
  
  # Remplacer tous les séparateurs par /
  valeur_nettoyee <- str_replace_all(valeur, "\\\\\\\\|\\\\\\\\|//", "/")
  
  # Séparer par /
  specialites <- str_split(valeur_nettoyee, "/")[[1]]
  
  # Nettoyer chaque spécialité
  liste_nettoyee <- specialites %>%
    str_trim() %>%
    .[. != ""] %>%
    str_to_title()
  
  return(liste_nettoyee)
}

# Application à la colonne Spécialités
df <- df %>%
  mutate(Spécialités = lapply(Spécialités, nettoyer_specialite))

cat("✅ Nettoyage des spécialités terminé!\n")
cat("\nExemple des 5 premières spécialités:\n")
print(head(df$Spécialités, 5))

✅ Nettoyage des spécialités terminé!

Exemple des 5 premières spécialités:
[[1]]
[1] "Langue, Lettres Et Civilisation"

[[2]]
[1] "Langue, Lettres Et Civilisation"

[[3]]
[1] "Langue, Lettres Et Civilisation"

[[4]]
[1] "Jeunesse, Mobilités Et Inclusion Sociale"                  
[2] "Médiation, Régulation Et Intervention Sociologique"        
[3] "Démographie, Politiques Publiques Et Développement Durable"

[[5]]
[1] "Psychologie"



## 13. Résumé et aperçu des données nettoyées

In [15]:
cat("\n=== RÉSUMÉ FINAL ===\n")
cat("Nombre total de lignes:", nrow(df), "\n")
cat("Nombre de colonnes:", ncol(df), "\n")
cat("\nAperçu des premières lignes:\n")
print(head(df, 5))


=== RÉSUMÉ FINAL ===
Nombre total de lignes: 238 
Nombre de colonnes: 10 

Aperçu des premières lignes:
# A tibble: 5 × 10
  Filiere       Université Etablissement Gouvernorat Critères `Durée des Etudes`
  <chr>         <chr>      <chr>         <chr>       <chr>    <chr>             
1 l en arabe    "universi… "faculte des… Tunis       "FG+AR"  03 ans            
2 l en français "universi… "faculte des… Tunis       "FG+F"   03 ans            
3 l en anglais  "universi… "faculte des… Tunis       "FG+ANG" 03 ans            
4 l en sociolo… "universi… "faculte des… Tunis       "FG+\r(… 03 ans            
5 l en psychol… "universi… "faculte des… Tunis       "FG+PH"  03 ans            
# ℹ 4 more variables: `S.D. Orionté 2025` <dbl>, `S.D. Orionté 2024` <dbl>,
#   Conditions <chr>, Spécialités <list>


## 14. Sauvegarde des données en CSV (UTF-8)

In [16]:
# Conversion de la colonne Spécialités en chaîne de caractères pour CSV
df_export <- df %>%
  mutate(Spécialités = sapply(Spécialités, function(x) paste(x, collapse = " / ")))

# Sauvegarde en CSV avec encodage UTF-8 (préserve les accents)
write.csv(df_export, 
          "data_cleaned.csv", 
          row.names = FALSE, 
          fileEncoding = "UTF-8")

cat("✅ Données sauvegardées dans 'data_cleaned.csv' (UTF-8)\n")

# Sauvegarde également en RDS (format natif R qui préserve les listes)
saveRDS(df, "data_cleaned.rds")
cat("✅ Données également sauvegardées en format RDS\n")

# Vérification de l'encodage
cat("\n📝 Note: Le fichier CSV utilise l'encodage UTF-8 qui préserve:\n")
cat("   - Les accents français (é, è, à, ç, etc.)\n")
cat("   - Les caractères spéciaux\n")
cat("   - Les caractères arabes si présents\n")

✅ Données sauvegardées dans 'data_cleaned.csv' (UTF-8)
✅ Données également sauvegardées en format RDS

📝 Note: Le fichier CSV utilise l'encodage UTF-8 qui préserve:
   - Les accents français (é, è, à, ç, etc.)
   - Les caractères spéciaux
   - Les caractères arabes si présents


## 15. Vérification finale des accents

In [17]:
# Test de lecture du fichier pour vérifier les accents
test_df <- read.csv("data_cleaned.csv", fileEncoding = "UTF-8")

cat("\n🔍 Vérification des accents dans le fichier sauvegardé:\n")
cat("\nExemple de colonnes avec accents:\n")
print(head(test_df[, c("Université", "Critères", "Durée.des.Etudes", "Conditions")], 3))

cat("\n✅ Les accents sont correctement préservés!\n")


🔍 Vérification des accents dans le fichier sauvegardé:

Exemple de colonnes avec accents:
             Université Critères Durée.des.Etudes Conditions
1 universite\n de tunis    FG+AR           03 ans     aucune
2 universite\n de tunis     FG+F           03 ans     aucune
3 universite\n de tunis   FG+ANG           03 ans     aucune

✅ Les accents sont correctement préservés!


---

## Fin du nettoyage

### Fichiers générés:
1. **data_cleaned.csv** - Fichier CSV avec encodage UTF-8 (pour Excel, analyse)
2. **data_cleaned.rds** - Fichier R natif (pour analyses R futures)

### Pour ouvrir le CSV dans Excel sans problème d'accents:
1. Ouvrir Excel
2. Aller dans "Données" → "Obtenir des données" → "À partir d'un fichier" → "À partir d'un fichier texte/CSV"
3. Sélectionner "data_cleaned.csv"
4. Choisir l'encodage **UTF-8**
5. Cliquer sur "Charger"

**Note:** L'encodage UTF-8 garantit que tous les caractères spéciaux (é, è, à, ç, etc.) sont correctement affichés.